## Pasnascope Pipeline

Integrates all of the modules from the package, to go from raw data to csv output.
More information about each module can be found in the other Jupyter Notebooks.
You should change the `experiment name` and the `img_path` at the fist cell.

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib widget
from pathlib import Path

import matplotlib.pyplot as plt
from matplotlib.collections import PatchCollection
from matplotlib.patches import Rectangle
from tifffile import imread

from pasnascope import activity, find_hatching, full_embryo_length, roi, slice_img, utils, vnc_length

experiment_name = '20240515'
root_dir = Path.cwd().parent
project_dir = root_dir.joinpath('data', experiment_name)

# Provide absolute paths for the raw image
img_path = '/home/cdp58/Documents/raw_images/pasnascope/20240515.tif'

If the raw data is in nd2 format, it must first be converted to a tif file, and that tif file should be used in the other cells of this jupyter notebook.

In [ ]:
img_path = '/home/cdp58/Documents/raw_images/pasnascope/20240515_c15525.nd2'
dest = '/home/cdp58/Documents/raw_images/pasnascope/20240515.tif'
should_convert = False

if should_convert:
    slice_img.save_as_tiff(img_path, dest)

Visualize the regions of the individual embryos that will be cropped.
If the bounding boxes don't fit the embryo, you can try changing the `thres_adjust` parameter at line 1.
The resulting image also shows the number that will be used for each embryo when saving the individual movies.

In [ ]:
coords = slice_img.calculate_slice_coordinates(img_path, n_cols=4, thres_adjust=0)
img = slice_img.get_first_image_from_mmap(img_path)

boundaries = coords.values()

rect_coords = [slice_img.boundary_to_rect_coords(b, img.shape) for b in boundaries]
recs = [Rectangle((y,x), w, h) for (x,y,w,h) in rect_coords]

centroids = {i: ((x0+x1)//2, (y0+y1)//2) for i, (x0, x1, y0, y1) in coords.items()}
h = img.shape[0]
w = img.shape[1]

# Visualize the slices before parsing the data:
# The img is equalized just to display the entire embryo
fig, ax = plt.subplots()
fig.canvas.header_visible = False
fig.canvas.resizable = False
ax.set_axis_off()

ax.imshow(img)

pc = PatchCollection(recs, color='red', alpha=0.7, linewidth=1, facecolor='none')
ax.add_collection(pc)

for i, (y, x) in centroids.items():
    ax.text(x, y, str(i))

plt.show()

If the image above looks good, change the variable `should_process` to `True` and run the next cell.
This will crop each embryo and save under the `data` directory. 

In [ ]:
dest = root_dir.joinpath('data', experiment_name, 'embs')
should_process = False

if should_process:
    dest.mkdir(parents=True, exist_ok=True)
    slice_img.cut_movies(coords, img_path, dest, overwrite=False)

Calculate the VNC length for each embryo, and save as a csv file in the `results` directory.

In [ ]:
interval = 20
img_dir = project_dir.joinpath('embs')
embs = sorted(img_dir.glob('*ch2.tif'), key=utils.emb_number)

dest = root_dir.joinpath('results', experiment_name, 'lengths')
dest.mkdir(parents=True, exist_ok=True)

lengths = []
ids = []

for emb in embs:
    print(emb.stem)
    hp = find_hatching.find_hatching_point(emb)
    hp -= hp % interval

    img = imread(emb, key=range(0,hp, interval))
    vnc_len = vnc_length.measure_VNC_centerline(img)
    lengths.append(vnc_len)
    id = utils.emb_number(emb.stem)
    ids.append(id)

saved = vnc_length.export_csv(ids, lengths, dest, interval)
if saved:
    print(f"Wrote length data for {len(embs)} embryos, at {dest}")

Calculate full embryo length.

In [ ]:
file_name = 'full-length.csv'

img_dir = project_dir.joinpath('embs')
embs = sorted(img_dir.glob('*ch2.tif'), key=utils.emb_number)

dest = root_dir.joinpath('results', experiment_name)
dest.mkdir(parents=True, exist_ok=True)
file_path = dest.joinpath(file_name)

lengths = []
embryo_names = []
if file_path.exists():
    print(f"The file {file_path.stem} already exists, and won't be overwritten. Change file or delete the existing file.")
else:
    for emb in embs:
        embryo_names.append(emb.stem)
        lengths.append(full_embryo_length.measure(emb))
    
    saved = full_embryo_length.export_csv(lengths, embryo_names, file_path)
    if saved:
        print(f"Wrote length data for {len(embs)} embryos, at {file_path}")
    

Calculate activity for active and structural channels and save as a csv file in the `results` directory.

In [ ]:
img_dir = project_dir.joinpath( 'embs')

# All structural channel movies end with the suffix ch2
active = sorted(img_dir.glob('*ch1.tif'), key=utils.emb_number)
struct = sorted(img_dir.glob('*ch2.tif'), key=utils.emb_number)

In [ ]:
dir_name = 'activity'
# The roi is calculated in steps of `window` size.
# This means that the slices from 0 to 10 will use the same 
# roi, that was calculated based on frame 0, for example.
window = 10

dest = root_dir.joinpath('results', experiment_name, dir_name)
dest.mkdir(parents=True, exist_ok=True)

embryos = []
ids = []
for act, stct in zip(active[:3], struct[:3]):
    id = utils.emb_number(act)
    file_path = dest.joinpath(f'emb{id}.csv')
    if file_path.exists():
        print(f'File {file_path.stem} already exists. Skipping..')
        continue
    active_img = imread(act)
    struct_img = imread(stct)
    mask = roi.get_roi(struct_img, window=window)

    masked_active = activity.apply_mask(active_img, mask)
    masked_struct = activity.apply_mask(struct_img, mask)

    signal_active = activity.get_activity(masked_active)
    signal_struct = activity.get_activity(masked_struct)

    emb = [signal_active, signal_struct]

    embryos.append(emb)
    ids.append(utils.emb_number(act))


saved = activity.export_csv(ids, embryos, dest)
if saved and len(embryos) > 0:
    print(f"Wrote activity data for {len(embryos)} embryos, at {dest}")
